In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
%matplotlib inline

# Stage 2

## Task 1

Compare the weekly statistics (Mean, Median, Mode) for cases and deaths across the US.

In [66]:
super_covid = pd.read_csv('./data/super_covid_data.csv')
covid_cases = pd.read_csv('./data/covid_confirmed_usafacts.csv')
covid_deaths = pd.read_csv('./data/covid_deaths_usafacts.csv')

# It will be easier to handle these separately so I'm importing the deaths and cases CSVs again.

# Create a dataframe I actually want to manipulate
cd_date = covid_deaths.drop(covid_deaths.columns[[0,1,2,3]], axis=1, inplace=False)

#Transpose
cd_date = cd_date.transpose()

# Rename the columns after the countyFIPS so that I can merge later.
cd_date.columns = covid_deaths['countyFIPS']

# Change the index to a date and time so that I can resample it.
cd_date.index = pd.to_datetime(cd_date.index)

#Repeat this process for Cases
cc_date = covid_cases.drop(covid_cases.columns[[0,1,2,3]], axis=1, inplace=False)
cc_date = cc_date.transpose()
cc_date.columns = covid_cases['countyFIPS']
cc_date.index = pd.to_datetime(cc_date.index)


# Resample and Assess
death_sums = cd_date.resample('W').sum()
death_mean = cd_date.resample('W').mean()

case_sums = cc_date.resample('W').sum()
case_mean = cc_date.resample('W').mean()

case_sums


,2020-01-26,2020-02-02,2020-02-09,2020-02-16,2020-02-23,2020-03-01,2020-03-08,2020-03-15,2020-03-22,2020-03-29,...,2023-05-21,2023-05-28,2023-06-04,2023-06-11,2023-06-18,2023-06-25,2023-07-02,2023-07-09,2023-07-16,2023-07-23
countyFIPS,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1001,0,0,0,0,0,0,0,0,0,29,...,139391,139391,19913,0,139391,139391,139391,139391,139391,139391
1003,0,0,0,0,0,0,0,2,11,46,...,493647,493647,70521,0,493647,493647,493647,493647,493647,493647
1005,0,0,0,0,0,0,0,0,0,0,...,53074,53074,7582,0,53074,53074,53074,53074,53074,53074
1007,0,0,0,0,0,0,0,0,0,0,...,57043,57043,8149,0,57043,57043,57043,57043,57043,57043
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56037,0,0,0,0,0,0,0,0,0,6,...,88280,88433,12635,0,88484,88495,88509,88515,88515,88515
56039,0,0,0,0,0,0,0,0,8,59,...,85173,85212,12174,0,85293,85334,85406,85442,85442,85442
56041,0,0,0,0,0,0,0,0,0,0,...,45169,45224,6462,0,45253,45262,45270,45276,45276,45276
